In [1]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import json
from sklearn.model_selection import train_test_split
import pandas as pd
import torch

# Load the data
#data = pd.read_csv('binary_labeled_text_data.csv')
f = open('chatgpt_responses.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)

df = pd.DataFrame(data['texts'])


df_ai = df['chatgpt']

df_human = df['human']

df_ai = pd.DataFrame(df_ai)

df_human = pd.DataFrame(df_human)

df_ai['label'] = 1

df_human['label'] = 0

df_ai = df_ai.rename(columns={'chatgpt': 'text'})
df_human = df_human.rename(columns={'human': 'text'})


data = pd.concat([df_ai, df_human])
df_shuffled = data.sample(frac=1).reset_index(drop=True)


In [2]:
import json
import pandas as pd  
import fasttext


df = df_shuffled

from sklearn.model_selection import train_test_split
""
train, test = train_test_split(df, test_size=0.2)


In [3]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from snowballstemmer import TurkishStemmer
  

nltk.download('stopwords')
nltk.download('punkt')

# Define Turkish stopwords
stop_words = set(stopwords.words('turkish'))

# Define Turkish stemmer
#stemmer = SnowballStemmer('turkish')
stemmer=TurkishStemmer()

def preprocess_text(text):
    # Tokenize the text into individual words
    tokens = word_tokenize(text.lower())

    # Remove stop words
    tokens = [token for token in tokens if token not in stop_words]

    # Stem the remaining words
    tokens = [stemmer.stemWord(token) for token in tokens]

    # Rejoin the words into a single string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bahadir/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/bahadir/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
df.columns = ['text','label']

In [5]:
from nltk.tokenize import word_tokenize
max_words = 200
df['text'] = df['text'].apply(preprocess_text)
df['text'] = df['text'].apply(lambda x: ' '.join(word_tokenize(x)[:max_words]))



In [6]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, stratify = df['label'])

In [7]:
data = []
for index, row in train.iterrows():
    label = '__label__' + str(row['label'])
    text = row['text']
    example = label + ' ' + text
    data.append(example)

# Save the examples to a text file
with open('chatgpt_fasttext_train_data.txt', 'w') as f:
    for example in data:
        f.write(example + '\n')
        
data = []
for index, row in test.iterrows():
    label = '__label__' + str(row['label'])
    text = row['text']
    example = label + ' ' + text
    data.append(example)

# Save the examples to a text file
with open('chatgpt_fasttext_test_data.txt', 'w') as f:
    for example in data:
        f.write(example + '\n')

In [8]:
import fasttext
# load the training data
train_data = "chatgpt_fasttext_train_data.txt"
# load the testing data
test_data = "chatgpt_fasttext_test_data.txt"

# train the model
model = fasttext.train_supervised(input=train_data, epoch=25, lr = 1.0)

# evaluate the model on the testing data
result = model.test(test_data)

# print the evaluation metrics
print("Number of examples:", result[0])
print("Precision:", result[1])
print("Recall:", result[2])



Read 0M words
Number of words:  36246
Number of labels: 2
Progress:  59.3% words/sec/thread: 4264895 lr:  0.407328 avg.loss:  0.292500 ETA:   0h 0m 0s

Number of examples: 1151
Precision: 0.7037358818418766
Recall: 0.7037358818418766


Progress: 100.0% words/sec/thread: 4255401 lr:  0.000000 avg.loss:  0.206039 ETA:   0h 0m 0s


In [9]:
model = fasttext.train_supervised(input=train_data, epoch=50, lr = 1.0)

# evaluate the model on the testing data
result = model.test(test_data)

# print the evaluation metrics
print("Number of examples:", result[0])
print("Precision:", result[1])
print("Recall:", result[2])


Read 0M words
Number of words:  36246
Number of labels: 2
Progress:  95.1% words/sec/thread: 4070595 lr:  0.048911 avg.loss:  0.127007 ETA:   0h 0m 0s

Number of examples: 1151
Precision: 0.7028670721112077
Recall: 0.7028670721112077


Progress: 100.0% words/sec/thread: 3894701 lr:  0.000000 avg.loss:  0.121953 ETA:   0h 0m 0s


In [10]:
model = fasttext.train_supervised(input=train_data, epoch=200, lr = 1)

# evaluate the model on the testing data
result = model.test(test_data)

# print the evaluation metrics
print("Number of examples:", result[0])
print("Precision:", result[1])
print("Recall:", result[2])

Read 0M words
Number of words:  36246
Number of labels: 2
Progress:  97.5% words/sec/thread: 3530382 lr:  0.024837 avg.loss:  0.047568 ETA:   0h 0m 0s

Number of examples: 1151
Precision: 0.7011294526498697
Recall: 0.7011294526498697


Progress: 100.0% words/sec/thread: 3474746 lr:  0.000000 avg.loss:  0.046341 ETA:   0h 0m 0s


In [14]:
len(train_data)

31

In [11]:
model = fasttext.train_supervised(input=train_data, epoch=200, lr = 0.1)

# evaluate the model on the testing data
result = model.test(test_data)

# print the evaluation metrics
print("Number of examples:", result[0])
print("Precision:", result[1])
print("Recall:", result[2])

Read 0M words
Number of words:  36246
Number of labels: 2
Progress:  95.8% words/sec/thread: 4173635 lr:  0.004191 avg.loss:  0.049152 ETA:   0h 0m 0s

Number of examples: 1151
Precision: 0.7124239791485665
Recall: 0.7124239791485665


Progress: 100.0% words/sec/thread: 4152610 lr:  0.000000 avg.loss:  0.047369 ETA:   0h 0m 0s


In [12]:
model = fasttext.train_supervised(input=train_data, epoch=2000, lr = 0.1)

# evaluate the model on the testing data
result = model.test(test_data)

# print the evaluation metrics
print("Number of examples:", result[0])
print("Precision:", result[1])
print("Recall:", result[2])

Read 0M words
Number of words:  36246
Number of labels: 2
Progress:  99.7% words/sec/thread: 4214394 lr:  0.000308 avg.loss:  0.010983 ETA:   0h 0m 0s

Number of examples: 1151
Precision: 0.7019982623805386
Recall: 0.7019982623805386


Progress: 100.0% words/sec/thread: 4207435 lr:  0.000000 avg.loss:  0.010956 ETA:   0h 0m 0s
